In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import string

In [3]:
df = pd.read_csv("spam ham data set.csv",encoding='latin-1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
df.dropna(axis=1, inplace=True)
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
trainSet = df[:3900]
trainSet

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
3895,spam,tells u 2 call 09066358152 to claim å£5000 pri...
3896,ham,No. Thank you. You've been wonderful
3897,ham,Otherwise had part time job na-tuition..
3898,ham,ÌÏ mean it's confirmed... I tot they juz say o...


In [6]:
testSet = df[3900:]
testSet.reset_index(inplace=True,drop=True)
testSet

,v1,v2
0,ham,That depends. How would you like to be treated...
1,ham,"Right on brah, see you later"
2,ham,Waiting in e car 4 my mum lor. U leh? Reach ho...
3,spam,Your 2004 account for 07XXXXXXXXX shows 786 un...
4,spam,Do you want a new video handset? 750 anytime a...
...,...,...
1667,spam,This is the 2nd time we have tried 2 contact u...
1668,ham,Will Ì_ b going to esplanade fr home?
1669,ham,"Pity, * was in mood for that. So...any other s..."
1670,ham,The guy did some bitching but I acted like i'd...


In [7]:
spamCount = 0
hamCount = 0
totalCount =  0
spamDict = defaultdict(int)
hamDict = defaultdict(int)
for index,row in trainSet.iterrows():
    totalCount+=1
    wordList = str(row['v2']).split()
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]
    wordList = [word for word in wordList if word != '']
    if str(row['v1'])=='spam':
        spamCount+=1
        addInDict = spamDict
    else:
        hamCount+=1
        addInDict = hamDict
    for word in wordList:
        addInDict[word] = addInDict[word]+1

print(totalCount)
print(spamCount)
print(hamCount)

3900
519
3381


In [8]:
pSpam = spamCount/totalCount
pHam = hamCount/totalCount
alpha = 1
k = 2
pWordSpamDict = {}
pWordHamDict = {}
for key in spamDict:
    pWordSpamDict[key] = (spamDict[key] + alpha)/(spamCount + (alpha*k))
for key in hamDict:
    pWordHamDict[key] = (hamDict[key] + alpha)/(hamCount + (alpha*k))

In [9]:
def spamProb(wordList):
    wordSpamProb = []
    for word in wordList:
        if word in pWordSpamDict:
            wordSpamProb.append(pWordSpamDict[word])
        else:
            p = (0 + alpha)/(spamCount + (alpha*k))
            wordSpamProb.append(p)
    prob = pSpam
    for item in wordSpamProb:
        prob*=item
    return prob

def hamProb(wordList):
    wordHamProb = []
    for word in wordList:
        if word in pWordHamDict:
            wordHamProb.append(pWordHamDict[word])
        else:
            p = (0 + alpha)/(hamCount + (alpha*k))
            wordHamProb.append(p)
    prob = pHam
    for item in wordHamProb:
        prob*=item
    return prob

In [10]:
actual = np.zeros(len(testSet.index), dtype=int)
predicted = np.zeros(len(testSet.index), dtype=int)
for index, row in testSet.iterrows():
    if str(row['v1'])=='spam':
        actual[index] = 1
    else:
        actual[index] = 0
    
    mail = str(row['v2'])
    wordList = str(mail).split()
    wordList = [(word.translate(str.maketrans('', '', string.punctuation))).lower() for word in wordList]
    wordList = [word for word in wordList if word != '']
    
    pSpamMail = spamProb(wordList)
    pHamMail = hamProb(wordList)
    
    if pSpamMail > pHamMail:
        predicted[index] = 1
    else:
        predicted[index] = 0
        
print('Total Test Mails : ' + str(len(testSet.index)))
print('Classified correctly : ' + str(np.sum(actual == predicted)))
print('Accuracy : ' + str(((np.sum(actual == predicted))/(len(testSet.index)))*100))

Total Test Mails : 1672
Classified correctly : 1271
Accuracy : 76.01674641148325


In [11]:
print(actual.shape)

(1672,)


In [12]:
print(predicted.shape)

(1672,)


In [13]:
a=0
b=0
c=0
d=0
for i in range(1672):
    if actual[i]==predicted[i]and actual[i]==0:
        a+=1
    if actual[i]!=predicted[i]and actual[i]==0:
        b+=1
    if actual[i]==predicted[i]and actual[i]==1:
        c+=1
    if actual[i]!=predicted[i]and actual[i]==1:
        d+=1

In [15]:
print(a,b)
print(c,d)

1043 401
228 0
